In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers --quiet

In [ ]:
!pip install --upgrade accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 41.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Oriserve/Reviews_Sentiments.csv")
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head()

,Review,Sentiment
0,Tires where delivered to the garage of my choi...,"['garage service positive', 'ease of booking p..."
1,"Easy Tyre Selection Process, Competitive Prici...","['garage service positive', 'value for money p..."
2,Very easy to use and good value for money.,['value for money positive']
3,Really easy and convenient to arrange,['ease of booking positive']
4,It was so easy to select tyre sizes and arrang...,"['location positive', 'value for money positiv..."


In [ ]:
print(df[df["Review"]=="test review, should be deleted"]['Sentiment'].iloc[0])

['refund not actioned positive', 'refund timescale positive', "mobile fitter didn't arrive positive", 'change of time positive', 'discount not applied positive', 'tyre agedot code positive', 'failed payment positive', 'late notice positive', 'facilities positive', 'incorrect tyres sent positive', 'call wait time positive', 'refund positive', 'no stock positive', 'balancing positive']


## It should be in the format -

#### [Q]
#### [A]

In [ ]:
with open('/content/drive/MyDrive/Oriserve/train_data.txt', 'w') as file:
    for index, row in df.iterrows():
        file.write(f"[Q] Identify the sentiment in the given sentence- {row['Review']}\n")
        file.write(f"[A] {row['Sentiment']}\n")

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPTNeoForCausalLM
from transformers import Trainer, TrainingArguments

In [ ]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPTNeoForCausalLM.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [ ]:
train_file_path = "/content/drive/MyDrive/Oriserve/train_data.txt"
model_name = 'EleutherAI/gpt-neo-125M'
output_dir = '/content/drive/MyDrive/Oriserve/GPT-neo'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 3
save_steps = 50000

In [ ]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,1.960500
1000,1.723400
1500,1.628000


In [ ]:
from transformers import PreTrainedTokenizerFast, GPTNeoForCausalLM, GPT2TokenizerFast, GPT2Tokenizer

In [ ]:
def load_model(model_path):
    model = GPTNeoForCausalLM.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    output = tokenizer.decode(final_outputs[0], skip_special_tokens=True)
    return output

In [ ]:
model_path = "/content/drive/MyDrive/Oriserve/GPT-neo"
max_len = 50

def final_answer(model_path, sequence, max_len):
  result = generate_text(model_path, sequence, max_len)
  result = result.split('[Q]')[0].strip()
  try:
    start_index = result.index('[A]')
    end_index = result.index('.', start_index) + 1
    result = result[:end_index]
  except:
    pass
  return result

In [ ]:
sequence1 = "Identify the sentiment in the given sentence- User friendly Website. Competitive Prices. Good communications. Efficient service by ATS Euromaster."
answer = final_answer(model_path, sequence1, max_len)
print(answer)

Identify the sentiment in the given sentence- User friendly Website. Competitive Prices. Good communications. Efficient service by ATS Euromaster. Good customer service.
[A] ['value for money positive']


In [ ]:
sequence1 = "Identify the sentiment in the given sentence- I had to wait three hours at the garage because they gave priority to customers with quicker jobs"
answer = final_answer(model_path, sequence1, max_len)
print(answer)

Identify the sentiment in the given sentence- I had to wait three hours at the garage because they gave priority to customers with quicker jobs.
[A] ['wait time negative']


In [ ]:
sequence1 = "Identify the sentiment in the given sentence-The company refunded my money in three days."
answer = final_answer(model_path, sequence1, max_len)
print(answer)

Identify the sentiment in the given sentence- The company refunded my money in three days.
[A] ['refund timescale positive', 'value for money positive']


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from transformers import GPTNeoForCausalLM, AutoTokenizer
from huggingface_hub import HfFolder, HfApi

# Load the model and tokenizer
model = GPTNeoForCausalLM.from_pretrained("/content/drive/MyDrive/Oriserve/GPT-neo")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Oriserve/GPT-neo")

# Define a model name for the Hub
model_name = "GPT-Neo-FineTuned"

# Push to Hugging Face Hub
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yashmathur0310/GPT-Neo-FineTuned/commit/7d3740998bc05ed840407c18224e595814312fd5', commit_message='Upload tokenizer', commit_description='', oid='7d3740998bc05ed840407c18224e595814312fd5', pr_url=None, pr_revision=None, pr_num=None)